<a href="https://colab.research.google.com/github/Rumeysakeskin/Speaker-Verification/blob/main/speaker_verification_with_torch_and_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
# BRANCH = 'main'
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
!pip install onnxruntime

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=89e3452cdf08f9a12118d66553f5a5c9b387e0f1dae54af27df80dedc5df91bc
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2build1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 16 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,760 kB of addit

EXPORT ONNX MODEL


In [2]:
import nemo.collections.asr as nemo_asr
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name="speakerverification_speakernet")
speaker_model.eval()
speaker_model.to('cpu')

# exporting pre-trained model to ONNX file for deployment.
speaker_model.export('speaker_model.onnx')

[NeMo I 2023-08-22 06:30:20 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/speakerverification_speakernet/versions/1.16.0/files/speakerverification_speakernet.nemo to /root/.cache/torch/NeMo/NeMo_1.21.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-08-22 06:30:21 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-08-22 06:30:21 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-08-22 06:30:21 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-08-22 06:30:21 features:289] PADDING: 16
[NeMo I 2023-08-22 06:30:22 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.


[NeMo W 2023-08-22 06:30:22 conv_asr:73] Turned off 20 masked convolutions
[NeMo W 2023-08-22 06:30:24 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input audio_signal
      warnings.warn(
    
[NeMo W 2023-08-22 06:30:24 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input length
      warnings.warn(
    
[NeMo W 2023-08-22 06:30:24 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2033: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input logits
      warnings.warn(
    
[NeMo W 2023-08-22 06:

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[NeMo I 2023-08-22 06:30:27 exportable:130] Successfully exported EncDecSpeakerLabelModel to speaker_model.onnx


(['speaker_model.onnx'],
 ['nemo.collections.asr.models.label_models.EncDecSpeakerLabelModel exported to ExportFormat.ONNX'])

SPEAKERNET ONNX INFERENCE

In [21]:
import time
import torch
import librosa
import pickle
import nemo
import onnxruntime
from ruamel.yaml import YAML
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import importlib
from omegaconf import OmegaConf
import tqdm
class NemoSpeakerIdentifier():
    def __init__(self):

        self._init_onnx_model()
        self._init_torch_model()

        # cosine similarity threshold: values between (-1,1)
        self.SIMILARITY_THRESHOLD = 0.4
        self.new_speaker_audio = None

        self.load_speakers_vectors()

    def _init_torch_model(self):

        self.verification_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name="speakerverification_speakernet")
        self.device = 'cpu'
        self.verification_model = self.verification_model.cpu()
        self.verification_model.eval()


    def _init_onnx_model(self):

        self.device = 'cpu'
        self.speakers_vectors_file = 'speakers_vectors_speakernet.pkl'
        speaker_model = 'speaker_model.onnx'
        self.sess = onnxruntime.InferenceSession(speaker_model)

        yaml = YAML(typ='safe')
        with open("model_config.yaml") as f:
            params = yaml.load(f)

        preprocessor_name = params['preprocessor']['_target_'].rsplit(".", 1)
        preprocessor_class = getattr(importlib.import_module(preprocessor_name[0]), preprocessor_name[1])
        preprocessor_config = params['preprocessor'].copy()
        preprocessor_config.pop('_target_')
        self.preprocessor = preprocessor_class(**preprocessor_config)

    def predict_speaker_vector(self, audio_file, mode):
       if mode == "torch":
        return self._torch_predict_speaker_vector(audio_file)
       if mode == "onnx":
        return self._onnx_predict_speaker_vector(audio_file)

    def _torch_predict_speaker_vector(self, audio_file):

      # default input sample rate is 16K
      audio, sr = librosa.load(audio_file, sr=16000)

      audio_length = audio.shape[0]
      audio_signal, audio_signal_len = (
          torch.tensor([audio], device=self.device),
          torch.tensor([audio_length], device=self.device),
      )
      _, embs = self.verification_model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
      emb_shape = embs.shape[-1]
      embs = embs.view(-1, emb_shape)

      return embs.cpu().detach().numpy()

    def _onnx_predict_speaker_vector(self, audio_file):

        # default input sample rate is 16K
        audio, sr = librosa.load(audio_file, sr=16000)

        audio_length = audio.shape[0]
        audio_signal, audio_signal_len = (
            torch.tensor([audio], device=self.device),  #torch.tensor([audio], device=self.device),
            torch.tensor([audio_length], device=self.device),
        )

        processed_signal, processed_signal_len = self.preprocessor(
            input_signal=audio_signal, length=audio_signal_len, )

        _, embs = self.sess.run(None, {self.sess.get_inputs()[0].name: processed_signal.tolist(),
                      self.sess.get_inputs()[1].name: processed_signal_len.tolist()})

        return embs

    def load_speakers_vectors(self):
        try:
            with open(self.speakers_vectors_file, 'rb') as f:
                self.speakers_vectors = pickle.load(f)

        except FileNotFoundError:
            self.speakers_vectors = {}

    def save_speakers_vectors(self):
        with open(self.speakers_vectors_file, 'wb') as f:
            pickle.dump(self.speakers_vectors, f)

    def add_speaker_vector(self, user_id, audio_file, mode):
        vector = self.predict_speaker_vector(audio_file, mode)
        self.speakers_vectors[user_id] = vector
        self.save_speakers_vectors()

    def get_most_similar_speaker(self, file, mode):

        self.consumed_time = []

        filepath = os.path.join("test_voices", file)

        if len(self.speakers_vectors) == 0:
            print("no speaker vector")

        start = time.time()
        vector = self.predict_speaker_vector(filepath, mode)

        speakers_vectors = [v[0] for v in self.speakers_vectors.values()]
        speakers_names = list(self.speakers_vectors.keys())

        sims = cosine_similarity([vector[0]], speakers_vectors)[0]
        end = time.time()
        print("Cosine similarity for {}: {}".format(file, sims))
        self.consumed_time.append(end-start)

        return self.consumed_time


import os
wav_files = os.listdir("./test_voices")

speaker_indentifier = NemoSpeakerIdentifier()

user_id = ["USER_1", "USER_2", "USER_3"]
user_ref_voices = ["./ref_voices/user1.wav", "./ref_voices/user2.wav", "./ref_voices/user3.wav"]

for id, voice in zip(user_id, user_ref_voices):
    speaker_indentifier.add_speaker_vector(id, voice, "torch")

for file in wav_files:
    speaker_indentifier.get_most_similar_speaker(file, "torch")

time_vector = speaker_indentifier.get_most_similar_speaker(file, "torch")
total_time = 0
for i in time_vector:
    total_time += i

print("Average consuming time for torch: {}\n".format(total_time / len(time_vector)))

for id, voice in zip(user_id, user_ref_voices):
    speaker_indentifier.add_speaker_vector(id, voice, "onnx")

for file in wav_files:
    speaker_indentifier.get_most_similar_speaker(file, "onnx")

time_vector = speaker_indentifier.get_most_similar_speaker(file, "onnx")
total_time = 0
for i in time_vector:
    total_time += i

print("Average consuming time for onnx: {}".format(total_time / len(time_vector)))

[NeMo I 2023-08-22 07:10:00 features:289] PADDING: 16
[NeMo I 2023-08-22 07:10:00 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.21.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
[NeMo I 2023-08-22 07:10:00 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.21.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-08-22 07:10:00 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-08-22 07:10:00 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-08-22 07:10:00 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-08-22 07:10:00 features:289] PADDING: 16
[NeMo I 2023-08-22 07:10:00 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
Cosine similarity for user3.wav: [0.4571687  0.39635766 0.99999994]
Cosine similarity for new_speaker_combined_audio.wav: [0.48274833 0.20362143 0.37188834]
Cosine similarity for new_speaker_combined_audio.wav: [0.48274833 0.20362143 0.37188834]
Average consuming time for torch: 0.54860520362854

Cosine similarity for user3.wav: [0.46686143 0.4067253  0.99995077]
Cosine similarity for new_speaker_combined_audio.wav: [0.47863206 0.20836455 0.3691328 ]
Cosine similarity for new_speaker_combined_audio.wav: [0.47882092 0.20900549 0.3700443 ]
Average consuming time for onnx: 0.368772029876709
